# OCR documentos en Python

## paso 1

se deben intalar las siguientes librerias

pip install opencv-python\
pip install pytesseract

In [116]:
from pdf2image import convert_from_path, convert_from_bytes
from pdf2image.exceptions import (
    PDFInfoNotInstalledError,
    PDFPageCountError,
    PDFSyntaxError
)

In [134]:
nomImg='alemania/Spiegel_0608_2300_33_Covid'
temp_dir='alemania/temp'

español='spa'
afr
amh
ara
asm
aze
aze-cyrl
bel
ben
bod
bos
bul
cat
ceb
ces
chi-sim
chi-tra
chr
cym
dan
dan-frak
alemania=deu
deu-frak
dev
dzo
ell
eng
enm
epo
est
eus
fas
fin
fra
frk
frm
gle
gle-uncial
glg
grc
guj
hat
heb
hin
hrv
hun
iku
ind
isl
ita
ita-old
jav
jpn
kan
kat
kat-old
kaz
khm
kir
koreano= kor
kur
lao
lat
lav
lit
mal
mar
mkd
mlt
msa
mya
nep
nld
nor
ori
pan
pol
portugues= por
pus
ron
rus
san
sin
slk
slk-frak
slv
spa
spa-old
sqi
srp
srp-latn
swa
swe
syr
tam
tel
tgk
tgl
tha
tir
tur
uig
ukr
urd
uzb
uzb-cyrl
vie
yid

In [135]:
idioma='deu'

In [136]:
images = convert_from_path(nomImg+'.pdf', dpi=150,  grayscale=True, output_folder=temp_dir)

In [137]:
temp_images = []
for i in range(len(images)):
            image_path = f'{temp_dir}/{i}.png'
            images[i].save(image_path, 'PNG')
            temp_images.append(image_path)




# paso 2 

preparamos la imagen para una mejor lectura vía ocr

In [138]:

# importing modules
import cv2
import csv
import pytesseract
from pytesseract import Output
# reading image using opencv
for i in range(len(images)):
    image = cv2.imread(f'{temp_dir}/{i}.png')
    print (i)
    #converting image into gray scale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # converting it to binary image by Thresholding
    # this step is require if you have colored image because if you skip this part 
    # then tesseract won't able to detect text correctly and this will give incorrect result
    threshold_img = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    #threshold_img = cv2.adaptiveThreshold(gray_image,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,11,2)
    # display image
    #cv2.imwrite(nomImg+'image.png', threshold_img)
    # Maintain output window until user presses a key
    cv2.waitKey(0)
    # Destroying present windows on screen
    cv2.destroyAllWindows()
    custom_config = r'--oem 3 --psm 6'
    # now feeding image to tesseract
    details = pytesseract.image_to_data(threshold_img, output_type=Output.DICT,  config=custom_config, lang=idioma)
    #print(details.keys())
    total_boxes = len(details['text'])
    for sequence_number in range(total_boxes):
        if int(details['conf'][sequence_number]) >30:
            (x, y, w, h) = (details['left'][sequence_number], details['top'][sequence_number], details['width'][sequence_number],  details['height'][sequence_number])
            threshold_img = cv2.rectangle(threshold_img, (x, y), (x + w, y + h), (0, 255, 0), 2)
    # display image
    #cv2.imwrite(nomImg+'image2.png', threshold_img)
    # Maintain output window until user presses a key
    cv2.waitKey(0)
    # Destroying present windows on screen
    cv2.destroyAllWindows()
    parse_text = []
    word_list = []
    last_word = ''
    for word in details['text']:
        if word!='':
            word_list.append(word)
            last_word = word
        if (last_word!='' and word == '') or (word==details['text'][-1]):
            parse_text.append(word_list)
            word_list = []
            
  
    with open(nomImg+'.txt',  'a', newline="") as file:
        csv.writer(file, delimiter=" ").writerows(parse_text)
        

0
1
2


#### desde aqui hacia abajo no


In [122]:
from pytesseract import Output

In [128]:

custom_config = r'--oem 3 --psm 6'
# now feeding image to tesseract
details = pytesseract.image_to_data(threshold_img, output_type=Output.DICT,  config=custom_config, lang=idioma)
#print(details.keys())

In [129]:
print(details.keys())

dict_keys(['level', 'page_num', 'block_num', 'par_num', 'line_num', 'word_num', 'left', 'top', 'width', 'height', 'conf', 'text'])


In [130]:
total_boxes = len(details['text'])
for sequence_number in range(total_boxes):
    if int(details['conf'][sequence_number]) >30:
        (x, y, w, h) = (details['left'][sequence_number], details['top'][sequence_number], details['width'][sequence_number],  details['height'][sequence_number])
        threshold_img = cv2.rectangle(threshold_img, (x, y), (x + w, y + h), (0, 255, 0), 2)
# display image
cv2.imwrite(nomImg+'image2.png', threshold_img)
# Maintain output window until user presses a key
cv2.waitKey(0)
# Destroying present windows on screen
cv2.destroyAllWindows()


In [131]:
parse_text = []
word_list = []
last_word = ''
for word in details['text']:
    if word!='':
        word_list.append(word)
        last_word = word
    if (last_word!='' and word == '') or (word==details['text'][-1]):
        parse_text.append(word_list)
        word_list = []

In [132]:
import csv
with open(nomImg+'.txt',  'w', newline="") as file:
    csv.writer(file, delimiter=" ").writerows(parse_text)

In [87]:
print (details['text'][302])

IndexError: list index out of range